# Задание «Предсказание карт внимания»

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import cv2
import tqdm
import torch
import imageio
import itertools
import torchvision
import numpy as np
from torch import nn
from torchinfo import summary
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from utils import normalize_map, padding, padding_fixation

[1713905211.322525] [cv-v100-common:683263:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


## 1. Подготовка данных

[Дублирование датасета с страницы задания на Google Диск для пользователей Colab [5.1 GB]](https://drive.google.com/file/d/1g801icIkktdOqLFo-sPMgOSrc-CRoNZi/view?usp=sharing)

In [3]:
class DatasetClass(Dataset):
    def __init__(self, path_data, transforms=None, num_frames=1, mode='val', val_steps=9, val_min_dist=40):
        self.input_path_data, self.gt_path_data = path_data
        self.num_frames = num_frames
        self.transforms = transforms
        self.mode = mode
        self.map_idx_to_video = [{'input': os.path.join(self.input_path_data, folder), 
                                  'gt': os.path.join(self.gt_path_data, folder)}
                                 for folder in sorted(os.listdir(self.input_path_data)) if 'x' not in folder]
        self.val_steps = val_steps
        self.val_min_dist = val_min_dist
        
    def __len__(self):
        return len(self.map_idx_to_video) * (self.val_steps if self.mode == 'val' else 1)

    def __getitem__(self, idx):
        
        
        if self.mode == 'val':
            # Берем первый кадр для валидации
            folder = self.map_idx_to_video[idx // self.val_steps]
            frames = sorted([x for x in os.listdir(os.path.join(folder['input'], 'frames')) if '.png' in x])
            
            start_idx = self.num_frames + self.val_min_dist  * (idx % self.val_steps)
        else:
            folder = self.map_idx_to_video[idx]
            frames = sorted([x for x in os.listdir(os.path.join(folder['input'], 'frames')) if '.png' in x])
            start_idx = np.random.randint(0, len(frames) - self.num_frames + 1)
        
        end_idx = start_idx + self.num_frames
        
        fragment = []
        for fname in frames[start_idx:end_idx]:
            frame = padding(
                        cv2.cvtColor(
                            cv2.imread(os.path.join(folder['input'], 'frames', fname)), 
                        cv2.COLOR_BGR2RGB)
                    ).astype('float32') / 255.
            fragment.append(self.transforms(frame))
        
        # Предсказываем карту внимания для последнего кадра в случайном подмножестве frames
        # Не забываем делать паддинг и нормализацию для единообразия
        saliency = normalize_map(padding(
                 cv2.imread(os.path.join(folder['gt'], 'gt_saliency', frames[end_idx - 1]), 
                 cv2.IMREAD_GRAYSCALE)))[np.newaxis].astype('float32')
        
        # Бейзлайн модель использует функцию потерь, основанную только на карте saliency
        # Но если понадобится, вы можете использовать и карты фиксаций
#         fixations = padding_fixation(
#                             cv2.imread(os.path.join(folder['gt'], 'gt_fixations', frames[end_idx - 1]), 
#                             cv2.IMREAD_GRAYSCALE))[np.newaxis].astype('float32')
        
        return fragment, saliency


# Бесконечное равномерное семплирование из датасета
class InfiniteSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, size):
        self.size = size

    def _infinite_indices(self):
        g = torch.Generator()
        while True:
            yield from torch.randperm(self.size, generator=g)

    def __iter__(self):
        yield from itertools.islice(self._infinite_indices(), 0, None, 1)

In [4]:
# Нормализация параметрами из ImageNet
normalization = torchvision.transforms.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])

denormalization = torchvision.transforms.Normalize(
                mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
                std=[1/0.229, 1/0.224, 1/0.225])

# Вы также можете попробовать использовать аугментации, но будьте аккуратны
# требуются преобразования, не меняющие области внимания, а также если вы используете
# несколько кадров, то пространственные преобразования между кадрами должны быть согласованы!
im_transform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    normalization
               ])

In [5]:
DATA_TRAIN = ['./tmp/01_test_file_input/train/', './tmp/01_test_file_gt/train/']
DATA_VAL = ['./tmp/01_test_file_input/test/', './tmp/01_test_file_gt/test/']

In [6]:
data_train = DatasetClass(DATA_TRAIN, transforms=im_transform, num_frames=5, mode='train')
data_valid = DatasetClass(DATA_VAL, transforms=im_transform, num_frames=5, mode='val')

train_loader = DataLoader(data_train, batch_size=8, sampler=InfiniteSampler(len(data_train)), 
                          num_workers=2, pin_memory=True)

valid_loader = DataLoader(data_valid, batch_size=1, shuffle=False, pin_memory=True)

## 3. Зададим функцию потерь

In [7]:
len(valid_loader)

27

Будем использовать дивергенцию [Кульбака-Лейблера](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence), сравнивая предсказанное распределение с эталонной картой внимания. Вы так же можете использовать другие функции потерь при обучении.

In [8]:
def kld(y_pred, y_true, eps=1e-7):
    """This function computes the Kullback-Leibler divergence between ground
       truth saliency maps and their predictions. Values are first divided by
       their sum for each image to yield a distribution that adds to 1.
    Args:
        y_true (tensor, float32): A 4d tensor that holds the ground truth
                                  saliency maps with values between 0 and 255.
        y_pred (tensor, float32): A 4d tensor that holds the predicted saliency
                                  maps with values between 0 and 1.
        eps (scalar, float, optional): A small factor to avoid numerical
                                       instabilities. Defaults to 1e-7.
    Returns:
        tensor, float32: A 0D tensor that holds the averaged error.
    """

    sum_true = torch.sum(y_true, dim=(1, 2, 3), keepdim=True)
    y_true = y_true / (eps + sum_true)

    sum_pred = torch.sum(y_pred, dim=(1, 2, 3), keepdim=True)
    y_pred = y_pred / (eps + sum_pred)
    
    loss = y_true * torch.log(eps + y_true / (eps + y_pred))
    loss = torch.mean(torch.sum(loss, dim=(1, 2, 3)))

    return loss


def cc_loss(s_map, gt):
    a = (s_map - torch.mean(s_map, axis=(1, 2, 3), keepdims=True)) / (torch.std(s_map, axis=(1, 2, 3), keepdims=True) + 1e-7)
    b = (gt - torch.mean(gt, axis=(1, 2, 3), keepdims=True)) / (torch.std(gt, axis=(1, 2, 3), keepdims=True) + 1e-7)
    r = torch.sum(a * b, axis=(1, 2, 3), keepdims=True) / torch.sqrt((a * a).sum(axis=(1, 2, 3), keepdims=True) * (b * b).sum(axis=(1, 2, 3), keepdims=True) + 1e-7)
    return -r.mean()

def sim_loss(s_map, gt):
    s_map = s_map / (torch.sum(s_map, axis=(1, 2, 3), keepdims=True) + 1e-7)
    gt = gt / (torch.sum(gt, axis=(1, 2, 3), keepdims=True) + 1e-7)
    r = torch.sum(torch.minimum(s_map, gt), axis=(1, 2, 3), keepdims=True)
    return -r.mean()

In [9]:
x = torch.ones((10, 20, 30, 40))
cc_loss(x, x).shape, sim_loss(x, x).shape

(torch.Size([]), torch.Size([]))

## 4. Обучим модель

__Пайплайн обучения__:
* Определить __таргет__. В нашей задаче это эталонная карта внимания
* Определить __функцию потерь (loss)__. Используем KLD из предыдущего этапа
* Выбрать __оптимизатор__. Чтобы всё быстро заработало, возьмем `Adam/AdamW` c [lr=3e-4](https://www.urbandictionary.com/define.php?term=Karpathy%20Constant)

In [49]:
from collections import defaultdict

In [15]:
len(next(iter(train_loader))[0])

5

In [11]:
from saliency_tased import SaliencyModel, SaliencyEvaluator

best_state_dict = None
best_val_loss = 1e10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SaliencyModel().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

summary(model, input_size=(8, 3, 216, 384))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [93]:
NUM_EPOCHS = 10
NUM_STEPS = 50

In [96]:
for epoch in range(NUM_EPOCHS):
    train_losses = defaultdict(list)
    valid_losses = defaultdict(list)

    # Часть слоев сети имеет два режима - train и eval
    # Важно переводить из одного в другой при обучении/тестировании
    model.train()

    ##################
    ### TRAIN LOOP ###
    ##################
    loop = tqdm.tqdm(itertools.islice(train_loader, NUM_STEPS), total=NUM_STEPS)
    for fragment, gt_saliency in loop:
        # Set to zero the parameter gradients for the current batch
        optimizer.zero_grad()

        # Prediction for one frame
        fragment = fragment[0].cuda()
        gt_saliency = gt_saliency.cuda()
        pred_saliency = model(fragment)

        # Calculate loss, make backward pass and update the parameters
        loss_kld = kld(pred_saliency, gt_saliency)
        loss_cc = cc_loss(pred_saliency, gt_saliency)
        loss_sim = sim_loss(pred_saliency, gt_saliency)
        loss = loss_kld # + loss_cc + loss_sim
        
        loss.backward()
        optimizer.step()

        loss_value = loss.item()
        loop.set_postfix(loss=loss_value)
        train_losses['total'].append(loss.item())
        train_losses['kld'].append(loss_kld.item())
        train_losses['cc'].append(loss_cc.item())
        train_losses['sim'].append(loss_sim.item())

    #######################
    ### VALIDATION LOOP ###
    #######################
    with torch.no_grad():
        model.eval()
        for fragment, gt_saliency in tqdm.tqdm(valid_loader, total=len(valid_loader)):
            optimizer.zero_grad()
            fragment = fragment[0].cuda()
            gt_saliency = gt_saliency.cuda()
            pred_saliency = model(fragment)

            loss_kld = kld(pred_saliency, gt_saliency)
            loss_cc = cc_loss(pred_saliency, gt_saliency)
            loss_sim = sim_loss(pred_saliency, gt_saliency)
            loss = loss_kld + loss_cc + loss_sim
            
            valid_losses["total"].append(loss.item())
            valid_losses["kld"].append(loss_kld.item())
            valid_losses["cc"].append(loss_cc.item())
            valid_losses["sim"].append(loss_sim.item())
            
        print(
            "| Epoch: {:4d}".format(epoch),
            "| Val Loss: {:.2f}".format(np.mean(valid_losses["total"])),
            "KLD: {:.2f}".format(np.mean(valid_losses["kld"])),
            "CC: {:.2f}".format(np.mean(valid_losses["cc"])),
            "SIM: {:.2f}".format(np.mean(valid_losses["sim"])),
            "| Train Loss: {:.2f}".format(np.mean(train_losses["total"])),
            "KLD: {:.2f}".format(np.mean(train_losses["kld"])),
            "CC: {:.2f}".format(np.mean(train_losses["cc"])),
            "SIM: {:.2f}".format(np.mean(train_losses["sim"])),
        )
        
        
        mean_val_loss = np.mean(valid_losses["total"])
        if mean_val_loss < best_val_loss:
            print('Updated ckpt')
            best_state_dict = model.state_dict()
            best_val_loss = mean_val_loss

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00, 13.48it/s]

| Epoch:    0 | Val Loss: -0.82 KLD: 0.56 CC: -0.74 SIM: -0.64 | Train Loss: -1.04 KLD: 0.41 CC: -0.78 SIM: -0.67



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 13.55it/s]

| Epoch:    1 | Val Loss: -0.91 KLD: 0.50 CC: -0.75 SIM: -0.65 | Train Loss: -1.07 KLD: 0.40 CC: -0.79 SIM: -0.68



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 13.65it/s]

...

| Epoch:    7 | Val Loss: -1.01 KLD: 0.46 CC: -0.79 SIM: -0.68 | Train Loss: -1.27 KLD: 0.31 CC: -0.84 SIM: -0.73
Updated ckpt



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 13.78it/s]

| Epoch:    8 | Val Loss: -0.98 KLD: 0.47 CC: -0.77 SIM: -0.68 | Train Loss: -1.25 KLD: 0.31 CC: -0.84 SIM: -0.73



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 13.90it/s]

| Epoch:    9 | Val Loss: -0.78 KLD: 0.59 CC: -0.73 SIM: -0.64 | Train Loss: -1.29 KLD: 0.29 CC: -0.85 SIM: -0.73


## 5. Протестируем модель

Сохраняем веса модели (этот файл **СДАЕТСЯ** в проверяющую систему)

In [97]:
torch.save(best_state_dict, 'ckpt/saliency_4.pth')

В качестве метрик будут использованы:
* Normalized Scanpath Saliency (NSS)
* Similarity score (SIM)
* Pearson’s Correlation Coefficient (CC)

[Подробнее про метрики](https://arxiv.org/pdf/1604.03605.pdf)

In [98]:
def nss(s_map, gt):
    x,y = np.where(gt)
    s_map_norm = (s_map - np.mean(s_map))/(np.std(s_map) + 1e-7)
    temp = []
    for i in zip(x,y):
        temp.append(s_map_norm[i[0], i[1]])
    return np.mean(temp)

In [99]:
def similarity(s_map, gt):
    s_map = s_map / (np.sum(s_map) + 1e-7)
    gt = gt / (np.sum(gt) + 1e-7)
    return np.sum(np.minimum(s_map, gt))

In [100]:
def cc(s_map, gt):
    a = (s_map - np.mean(s_map))/(np.std(s_map) + 1e-7)
    b = (gt - np.mean(gt))/(np.std(gt) + 1e-7)
    r = (a*b).sum() / np.sqrt((a*a).sum() * (b*b).sum() + 1e-7)
    return r

Помимо описания класса модели в **СДАВАЕМОМ** файле `saliency.py` необходимо описать класс `SaliencyEvaluator`, выполняющий логику загрузки модели и инференса на заданном видео с сохранением предсказаний.

Функция принимает путь до входных последовательностей кадров и путь до выходной папки предсказаний.

В этой функции нужно описать процесс загрузки весов модели получение карт внимания для всех входных видеопоследовательностей. Если ваша модель использует более одного кадра или требует дополнительных преобразований входа, реализуйте их внутри этой функции.

In [101]:
DATA_VAL[0]

'./tmp/01_test_file_input/test/'

In [102]:
import os

# Делаем предсказания моделью для всех входных кадров видео
evaluator = SaliencyEvaluator(model_path='ckpt/saliency_4.pth')

for video_name in sorted(os.listdir(DATA_VAL[0])):

    video_frames_path = os.path.join(DATA_VAL[0], video_name, 'frames')
    output_saliency_path = os.path.join('./outputs', video_name)
    os.makedirs(output_saliency_path, exist_ok=True)

    evaluator.evaluate(video_frames_path, output_saliency_path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 450/450 [00:33<00:00, 13.35it/s]


In [103]:
# Функция вычисления тестовых метрик
from utils import calculate_metrics

In [ ]:
calculate_metrics('./outputs/', DATA_VAL[1])

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍   | 440/450 [00:09<00:00, 47.20it/s]

In [ ]:
# Аналогичный вызов, которым будет тестироваться ваша модель в проверочной системе
# !python3 run.py ./public_tests/

In [ ]:
# Сделаем анимацию полученной карты внимания
def gif_creator(input_frames_path, predictions_path, gif_name='out.gif'):

    with imageio.get_writer(gif_name, mode="I") as writer:
        # Берем каждый 5ый кадр для ускорения
        for input_name in tqdm.tqdm(sorted(os.listdir(input_frames_path)[::5])):
            if '.png' not in input_name:
                continue
                
            pred_sm = cv2.imread(os.path.join(predictions_path, input_name), 0)
            # Invertion for colormap
            norm_sm = ((1 - normalize_map(pred_sm)) * 255).astype(np.uint8)
            heatmap = cv2.applyColorMap(norm_sm, cv2.COLORMAP_JET)
            frame = cv2.imread(os.path.join(input_frames_path, input_name))[:, :, ::-1]
            
            alpha = 0.7
            alpha_blended = np.clip(alpha * frame + (1 - alpha) * heatmap, 0, 255).astype(np.uint8)
            
            writer.append_data(alpha_blended)

In [ ]:
gif_creator(input_frames_path=os.path.join(DATA_VAL[0], '00/frames'),
            predictions_path=os.path.join('./outputs/00/'))

In [ ]:
from IPython.display import Image

Image(open('out.gif','rb').read())

## 6. Дальнейшие шаги

Для улучшения качества вашей модели можно попробовать следующие этапы:
* Сделать более информативную валидацию, например, добавив подсчет тестовых метрик прямо в validation loop
* Попробовать обучать большее число эпох, использовать регуляризацию, например, через Dropout
* Добавить skip-connection в Encoder-Decoder
* Сделать аугментации ([albumentations](https://github.com/albumentations-team/albumentations)). Будьте аккуранты с преобразованиями, которые могут потенциально изменить эталонные карты внимания. Начать можно с горизонтальных отражений
* Попробовать другие архитектуры, функции потерь и стратегии обучения
* Использовать информацию из более чем одного кадра (3D Conv/LSTM/GRU/любой другой способ агрегации). Обратите внимание: даже если ваш метод требует окно из кадров, тестирование всё равно будет учитывать все кадры видео. Во время тестирования вы можете искуственно дублировать первый кадр для накопления нужной ширины окна для предсказания.
* Поискать методы, решающие похожие задачи
* Пофантазировать и вдохновиться